In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import string

# let's keep our keras backend tensorflow quiet
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
# for testing on CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Model
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import numpy as np
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint


In [2]:
csv_File_Path = r'C:\Users\Sachin\Desktop\home\A_Z Handwritten Data.csv'
model_File_Path = r'C:\Users\Sachin\Desktop\home\Keras_Alphabet_Model.h5'

In [8]:
df = pd.read_csv(csv_File_Path)

In [9]:
X = df.drop(df.columns[0],axis = 1)
y = df[df.columns[0]]

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [11]:
print("X_train shape", X_train.shape)
print("y_train shape", Y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", Y_test.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# print the final input shape ready for training
print("Train matrix shape", X_train.shape)
print("Test matrix shape", X_test.shape)

X_train shape (260715, 784)
y_train shape (260715,)
X_test shape (111735, 784)
y_test shape (111735,)
Train matrix shape (260715, 784)
Test matrix shape (111735, 784)


In [12]:
print(np.unique(Y_train, return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64), array([ 9733,  6090, 16220,  7161,  8030,   828,  4029,  5066,   775,
        6002,  3894,  8163,  8655, 13440, 40348, 13502,  4039,  8139,
       33979, 15736, 20265,  2884,  7504,  4367,  7613,  4253],
      dtype=int64))


In [13]:
n_classes = 26
print("Shape before one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (260715,)


In [14]:
Y_train = np_utils.to_categorical(Y_train, n_classes )
Y_test = np_utils.to_categorical(Y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape after one-hot encoding:  (260715, 26)


In [25]:
n_classes = 26

In [26]:
checkpoint = ModelCheckpoint(model_File_Path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [39]:
model = Sequential()
model.add(Dense(1024, input_shape=(784,)))
model.add(Activation('relu'))                            
model.add(Dropout(0.2))

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [40]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [41]:
checkpoint = ModelCheckpoint(model_File_Path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [42]:
model.fit(X_train, Y_train,
          batch_size=128, epochs=2,
          verbose=2,
          validation_data=(X_test, Y_test))

Train on 260715 samples, validate on 111735 samples
Epoch 1/2
 - 171s - loss: 0.2240 - acc: 0.9358 - val_loss: 0.1134 - val_acc: 0.9660
Epoch 2/2
 - 164s - loss: 0.1027 - acc: 0.9694 - val_loss: 0.0755 - val_acc: 0.9781


In [44]:
model.save(model_File_Path)

In [45]:
new_model = load_model(model_File_Path)

In [46]:
assert_allclose(model.predict(X_train),
                new_model.predict(X_train),
                1e-5)

In [47]:
checkpoint = ModelCheckpoint(model_File_Path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [48]:
new_model.fit(X_train, Y_train,
          batch_size=128, epochs=2,
          verbose=2,
          validation_data=(X_test, Y_test))

Train on 260715 samples, validate on 111735 samples
Epoch 1/2
 - 162s - loss: 0.0796 - acc: 0.9757 - val_loss: 0.0687 - val_acc: 0.9807
Epoch 2/2
 - 163s - loss: 0.0669 - acc: 0.9796 - val_loss: 0.0636 - val_acc: 0.9823


In [49]:
new_model.save(model_File_Path)